In [547]:
import tweepy #https://github.com/tweepy/tweepy
import csv

#Twitter API credentials
consumer_key = "fRSNuw39bpDfHl2ybG28CT0JA"
consumer_secret = "gAZ5X5e1hJL6MJC7Tu8u0PHMVeYcDsv5VLVS31CehD6VNPcRtk"
access_key = "539175684-5RWLMw8Hyjc7CcpXyX5knwxlxfi3l7wFbqvpxN7F"
access_secret = "8U6D6rzn7zbrKkmgkni3XfiWvXlWBimdN5mMR5vE9MojD"


def get_all_tweets(screen_name):
	#Twitter only allows access to a users most recent 3240 tweets with this method
	
	#authorize twitter, initialize tweepy
	auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
	auth.set_access_token(access_key, access_secret)
	api = tweepy.API(auth)
	
	#initialize a list to hold all the tweepy Tweets
	alltweets = []	
	
	#make initial request for most recent tweets (200 is the maximum allowed count)
	new_tweets = api.user_timeline(screen_name = screen_name,count=200)
	#print(new_tweets[0])
	#save most recent tweets
	alltweets.extend(new_tweets)
	
	#save the id of the oldest tweet less one
	oldest = alltweets[-1].id - 1
	
	#keep grabbing tweets until there are no tweets left to grab
	while len(new_tweets) > 0:
		print ("getting tweets before %s" % (oldest))
		
		#all subsiquent requests use the max_id param to prevent duplicates
		new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)
		
		#save most recent tweets
		alltweets.extend(new_tweets)
		
		#update the id of the oldest tweet less one
		oldest = alltweets[-1].id - 1
		
		print ("...%s tweets downloaded so far" % (len(alltweets)))
	
	#transform the tweepy tweets into a 2D array that will populate the csv	
	try:
		outtweets = [[tweet.id_str, tweet.created_at, tweet.text] for tweet in alltweets]
        
	except:
		pass
	
	#write the csv	
# 	with open('%s_tweets.csv' % screen_name, 'w') as f:
# 		writer = csv.writer(f)
# 		writer.writerow(["id","created_at","text"])
# 		writer.writerows(outtweets)
	
	pass
	return outtweets


if __name__ == '__main__':
	#pass in the username of the account you want to download
	outtweets = get_all_tweets("imrankhanpti")

getting tweets before 899898376258375680
...400 tweets downloaded so far
getting tweets before 874258442566893571
...600 tweets downloaded so far
getting tweets before 844855696260644869
...800 tweets downloaded so far
getting tweets before 819828131099971583
...1000 tweets downloaded so far
getting tweets before 785531490083991551
...1200 tweets downloaded so far
getting tweets before 753523932993622015
...1400 tweets downloaded so far
getting tweets before 715779322792001535
...1600 tweets downloaded so far
getting tweets before 687341693696344064
...1800 tweets downloaded so far
getting tweets before 662968362104918015
...2000 tweets downloaded so far
getting tweets before 625188327000424448
...2200 tweets downloaded so far
getting tweets before 592598889041043455
...2400 tweets downloaded so far
getting tweets before 554639542717018112
...2600 tweets downloaded so far
getting tweets before 510843767470194687
...2800 tweets downloaded so far
getting tweets before 483360208048365567


In [2]:
import pandas as pd
tweets_df = pd.DataFrame(outtweets)

NameError: name 'outtweets' is not defined

In [549]:
tweets_df.to_csv('imrankhanpti.csv',index = False)

In [1]:
import gensim
import logging
import os
import nltk.data
import string
%matplotlib inline
import spacy
nlp = spacy.load('xx')

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
import pandas as pd
tweets_df = pd.read_csv('imrankhanpti.csv',encoding = 'ISO-8859-1')

In [4]:
article = []
texts = []
lines = []
#file = open('imran_khan_corpus.txt', 'wb')
for tweet in tweets_df['2']:
    tweet = nlp(tweet)
    for w in tweet:
        if w.text != '\n' and not w.is_stop and not w.is_punct and not w.like_num and 'RT' not in w.text and not w.like_url:
        # we add the lematized version of the word
            article.append(w.lemma_.lower())
    tweet = article
    to_be_written = ' '.join(article)
    lines.append(to_be_written)
    texts.append(tweet)
    article = []
#file.writelines(lines)
#file.close()

In [546]:
file = open('imran_khan_corpus.txt', 'r')
file.readline()

''

In [561]:
lines

['today chishtian distt bahawalnagar rural pakistan aware cancer corruption urban centres t',
 'a message youth pakistan',
 'the reformed professional kp police force special rapid response unit difference dealing wit',
 'saddened condemnable terrorist attack peshawar morning my prayers victims families',
 'greetings muslims eid milad un nabi sal lalla hu elyhy wasullam pbuh',
 'after pm times ns not know public holders explain assets expenditures ar',
 "another step forward pti 's commitment women rights first time kp open kutchery women wom",
 'in addition strict enforcement anti corruption laws societies fear social boyc',
 'how china deals corrupt contrast ruling elite protects rob public presently',
 'first report yrs hc sc praised police phc commends igp kp amp kp police handlin',
 'my statement today asking resignation pm interior minister',
 'my statement issued yesterday',
 "hafizabad junoon today they knew answer corruption i repeated ns 's wailing question mujhe",
 'saddened

In [327]:
bigram = gensim.models.Phrases(texts)


In [320]:
model = gensim.models.Word2Vec(iter=50, min_count=5, size = 200, sg = 1, window = 5,workers = 4)
model.build_vocab(texts, progress_per=10000)

In [321]:
model.train(texts, total_examples = model.corpus_count, epochs=model.iter, report_delay = 1)

1338818

In [326]:
model.most_similar('politics')

[('fear', 0.5887734889984131),
 ('exploitation', 0.5495667457580566),
 ('mafias', 0.5490497946739197),
 ('parties', 0.5414113998413086),
 ('mafia', 0.5330653190612793),
 ('armed', 0.5302530527114868),
 ('mindset', 0.5233873724937439),
 ('tactics', 0.5186854600906372),
 ('sh', 0.5161311626434326),
 ('pro', 0.4945696294307709)]

In [255]:
round(word_vectors.shape[0]/5)

338

In [290]:
from sklearn.cluster import KMeans
import time

start = time.time() # Start time

# Set "k" (num_clusters) to be 1/5th of the vocabulary size, or an
# average of 5 words per cluster
word_vectors = model.wv.syn0norm
num_clusters = round(word_vectors.shape[0] / 5)

# Initalize a k-means object and use it to extract centroids
kmeans_clustering = KMeans( n_clusters = num_clusters )
idx = kmeans_clustering.fit_predict(word_vectors)

# Get the end time and print how long the process took
end = time.time()
elapsed = end - start
print("Time taken for K Means clustering: ", elapsed, "seconds.")

Time taken for K Means clustering:  29.5917010307312 seconds.


In [277]:
kmeans_clustering?

In [292]:
word_centroid_map = dict(zip( model.wv.index2word, idx ))

In [293]:
cluster

279

In [296]:
for cluster in range(0,50):
    #
    # Print the cluster number  
    print("\nCluster %d" % cluster)
    #
    # Find all of the words for that cluster number, and print them out
    words = []
    for i in range(0,len(word_centroid_map.values())):
        if( list(word_centroid_map.values())[i] == cluster ):
            words.append(list(word_centroid_map.keys())[i])
    print(words)


Cluster 0
['involved', 'investigation']

Cluster 1
['victory', 'defeat', 'awakened', 'assemblies', 'eradicate', 'belongs', 'hr']

Cluster 2
['use', 'gas', '+', 'tear', 'rubber', 'bullets', 'excessive', 'tenures', 'shells', 'lethal', 'expired']

Cluster 3
['dr', 'devastated', 'shahid', 'university', 'javed', 'her', 'azhar', 'hanif', 'enjoyed', 'english', 'dera', 'smt', 'contribution', 'cousin']

Cluster 4
['bn', 'losses', 'laundered', 'yearly', 'smuggling', 'bns', 'estate', 'doubled', '10bn', 'gop', 'palace', 'fdi', 'audited']

Cluster 5
['lawyer', 'exposes', 'judges', 'admits', 'rejected', 'trustee', 'detailed', 'declaring', 'file', 'ownership', 'suing', 'refusing', 'hysterical', 'discredit', 'bench', 'maryum']

Cluster 6
['break', 'index', 'erdogan', 'delivered']

Cluster 7
['rid', 'civil', 'iran', 'sanctions', 'nuclear']

Cluster 8
['innocent', 'martyred', 'tragedy', 'nwa', 'daily', 'killed', 'civilians', 'hrs', 'heartbreaking', 'disrespect', 'displaced']

Cluster 9
['strength', 'ba

In [ ]:
model.syn0_lockf

In [52]:
texts, article = [], []
for w in doc:
    # if it's not a stop word or punctuation mark, add it to our article!
    if w.text != '\n' and not w.is_stop and not w.is_punct and not w.like_num:
        # we add the lematized version of the word
        article.append(w.lemma_)
    # if it's a new line, it means we're onto our next document
    if w.text == '\n':
        texts.append(article)
        article = []

,0,1,2
0,935590191594639363,2017-11-28 19:24:18,RT @HistoryToLearn: Berlin after world war 2 h...
1,934883988984025088,2017-11-26 20:38:06,"Insha Ji!\nUtho, ab kooch karo, iss sheher mei..."
2,934872097511309312,2017-11-26 19:50:51,RT @ZarrarKhuhro: This is us right now. That'l...
3,934684046302597120,2017-11-26 07:23:36,RT @KhaledBeydoun: The terror attack on the Si...
4,934368978088136704,2017-11-25 10:31:38,RT @Maria_Memon: رحمتِ سیدِؐ لولاک پہ کامل ایم...
5,933398093558665216,2017-11-22 18:13:41,"RT @ajplus: His soldiers slaughtered 8,000 Bos..."
6,933398056376074240,2017-11-22 18:13:32,"RT @AJEnglish: The “Butcher of Bosnia”, Rakto ..."
7,933011121753219073,2017-11-21 16:36:00,RT @SkyNews: Robert Mugabe has resigned as Pre...
8,932326908842397698,2017-11-19 19:17:11,Laazim hai ke hum bhi dekhengay\n#TinaSani #Fa...
9,930404025106882562,2017-11-14 11:56:20,RT @NadeemfParacha: Rare clip of Karachi in 19...


In [328]:
texts = [bigram[line] for line in texts]

C:\ProgramData\Anaconda3\lib\site-packages\gensim\models\phrases.py:431: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [330]:
from gensim.corpora import Dictionary
dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [342]:
from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim.models.wrappers import LdaMallet
from gensim.corpora import Dictionary
import pyLDAvis.gensim

In [334]:
hdpmodel = HdpModel(corpus=corpus, id2word=dictionary)

In [335]:
hdpmodel.show_topics()

[(0,
  "0.003*amp + 0.001*'s + 0.001*last + 0.001*eye + 0.001*entire + 0.001*harder + 0.001*resolute + 0.001*pakistan + 0.001*hardly + 0.001*moist + 0.001*pm + 0.001*accounts + 0.001*nayapakistan + 0.001*pc + 0.001*nation + 0.001*bhus + 0.001*venue + 0.001*thrashed + 0.001*ruler + 0.001*aggravated"),
 (1,
  "0.002*amp + 0.001*'s + 0.001*generously + 0.001*3,'pindi + 0.001*markhor + 0.001*vote + 0.001*pti + 0.001*send + 0.001*orders + 0.001*church + 0.001*parliament + 0.001*pmln + 0.001*cake + 0.001*dawnleak + 0.001*courts + 0.001*release + 0.001*william + 0.001*rs.12 + 0.001*pti_workers + 0.001*inauguration"),
 (2,
  "0.002*amp + 0.001*'s + 0.001*i + 0.001*rubber + 0.001*attention + 0.001*turnout + 0.001*corruption + 0.001*bodes + 0.001*shd + 0.001*world + 0.001*bec + 0.001*nishani + 0.001*lovers + 0.001*check + 0.001*pmln + 0.001*rs13 + 0.001*poodle + 0.001*electiion + 0.001*صبح + 0.001*expressed"),
 (3,
  "0.002*amp + 0.002*'s + 0.001*fearless + 0.001*ary + 0.001*bluffing + 0.001*lav

In [338]:
ldamodel = LdaModel(corpus=corpus, num_topics=20, id2word=dictionary)

In [339]:
ldamodel.show_topics()

[(5,
  '0.032*"amp" + 0.020*"pti" + 0.019*"i" + 0.013*"\'s" + 0.010*"pakistanis" + 0.008*"govt" + 0.007*"allah" + 0.007*"form" + 0.006*"aps" + 0.005*"pakistan"'),
 (19,
  '0.030*"amp" + 0.023*"\'s" + 0.013*"kp" + 0.011*"pti" + 0.008*"govt" + 0.008*"donors" + 0.007*"pakistan" + 0.006*"fateha" + 0.005*"police" + 0.005*"in"'),
 (14,
  '0.014*"amp" + 0.013*"pak" + 0.011*"today" + 0.010*"the" + 0.010*"r" + 0.008*"govt" + 0.007*"pti" + 0.007*"\'s" + 0.007*"students" + 0.006*"constituencies"'),
 (6,
  '0.058*"amp" + 0.013*"rigging" + 0.012*"\'s" + 0.011*"pti" + 0.008*"i_want" + 0.008*"pmln" + 0.006*"not" + 0.006*"geo" + 0.006*"polling" + 0.005*"the"'),
 (17,
  '0.027*"amp" + 0.017*"\'s" + 0.010*"workers" + 0.009*"pmln" + 0.009*"pti" + 0.009*"rigging" + 0.009*"we" + 0.008*"govt" + 0.007*"dharna" + 0.006*"sindh"'),
 (4,
  '0.050*"r" + 0.033*"amp" + 0.010*"i" + 0.010*"govt" + 0.008*"we" + 0.008*"pti" + 0.007*"today" + 0.006*"\'s" + 0.005*" " + 0.005*"ppl"'),
 (1,
  '0.025*"amp" + 0.013*"\'s" + 0

In [340]:
lsimodel = LsiModel(corpus=corpus, num_topics=10, id2word=dictionary)
lsimodel.show_topics(num_topics=5)  # Showing only the top 5 topics


[(0,
  '0.911*"amp" + 0.228*"\'s" + 0.129*"pti" + 0.087*"govt" + 0.078*"i" + 0.073*"pak" + 0.072*"pmln" + 0.060*"kp" + 0.059*"ppl" + 0.059*"today"'),
 (1,
  '-0.835*"\'s" + 0.334*"amp" + -0.307*"pti" + -0.137*"pmln" + -0.108*"kp" + -0.074*"govt" + -0.069*"pak" + -0.066*"today" + -0.051*"ns" + -0.049*"i"'),
 (2,
  '0.803*"pti" + -0.405*"\'s" + 0.242*"kp" + 0.188*"govt" + 0.092*"pmln" + 0.085*"i" + -0.079*"amp" + -0.058*"pak" + -0.057*"ns" + 0.051*" "'),
 (3,
  '0.862*"i" + 0.246*"today" + -0.147*"govt" + 0.135*"pak" + -0.124*"pti" + 0.112*"pakistan" + -0.092*"amp" + -0.089*"\'s" + 0.078*"r" + -0.071*"pmln"'),
 (4,
  '0.749*"govt" + 0.291*"pak" + -0.285*"pti" + 0.284*"pmln" + -0.132*"\'s" + 0.130*"ppl" + -0.114*"amp" + 0.112*"corruption" + 0.085*"the" + 0.083*"shameful"')]

In [343]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
8      6.728987        1       1  0.020699 -0.032483
4      6.230333        1       2  0.013333 -0.099910
13     6.193635        1       3  0.043001  0.035426
6      5.751859        1       4 -0.013063 -0.000660
2      5.534917        1       5 -0.046206 -0.013878
17     5.534769        1       6  0.042533  0.099613
16     5.282774        1       7  0.053984 -0.039154
5      5.223192        1       8  0.005404 -0.042366
0      5.220189        1       9 -0.054794 -0.051852
9      5.169812        1      10  0.092393 -0.059863
14     5.138620        1      11  0.036873 -0.044337
10     4.721251        1      12 -0.115532 -0.106385
19     4.518583        1      13  0.036853 -0.004052
7      4.327673        1      14 -0.056477  0.067217
1      4.258506        1      15 -0.032215  0.036389
11     4.227351        1      16 -0.166636  0.054440
12     4.134596        1      17 -0.003862  0.046069
3      3.950912        1      18  0.082663 -0.031970
15     3.935368        1      19  0.050426  0.125271
18     3.916673        1      20  0.010623  0.062486, topic_info=     Category         Freq           Term        Total  loglift  logprob
term                                                                    
127   Default   152.000000              r   152.000000  30.0000  30.0000
4485  Default    43.000000           idps    43.000000  29.0000  29.0000
4669  Default    37.000000          nadra    37.000000  28.0000  28.0000
25    Default    34.000000            eid    34.000000  27.0000  27.0000
1529  Default    54.000000  naya_pakistan    54.000000  26.0000  26.0000
1750  Default    86.000000        rigging    86.000000  25.0000  25.0000
4644  Default    32.000000          polio    32.000000  24.0000  24.0000
105   Default   224.000000              i   224.000000  23.0000  23.0000
57    Default   162.000000             kp   162.000000  22.0000  22.0000
50    Default   327.000000            pti   327.000000  21.0000  21.0000
2169  Default    41.000000             jc    41.000000  20.0000  20.0000
3671  Default    24.000000       dialogue    24.000000  19.0000  19.0000
561   Default    44.000000     inshaallah    44.000000  18.0000  18.0000
648   Default    27.000000           form    27.000000  17.0000  17.0000
172   Default   128.000000         people   128.000000  16.0000  16.0000
218   Default    54.000000          peace    54.000000  15.0000  15.0000
324   Default    55.000000       election    55.000000  14.0000  14.0000
2575  Default    49.000000        workers    49.000000  13.0000  13.0000
343   Default   161.000000            pak   161.000000  12.0000  12.0000
296   Default    32.000000        tsunami    32.000000  11.0000  11.0000
51    Default   497.000000             's   497.000000  10.0000  10.0000
4293  Default    16.000000            ikf    16.000000   9.0000   9.0000
2915  Default    35.000000         dharna    35.000000   8.0000   8.0000
346   Default    65.000000      elections    65.000000   7.0000   7.0000
24    Default    25.000000        muslims    25.000000   6.0000   6.0000
439   Default    30.000000        killing    30.000000   5.0000   5.0000
90    Default  1211.000000            amp  1211.000000   4.0000   4.0000
3     Default   160.000000       pakistan   160.000000   3.0000   3.0000
2698  Default    18.000000   timber_mafia    18.000000   2.0000   2.0000
2450  Default    27.000000          insaf    27.000000   1.0000   1.0000
...       ...          ...            ...          ...      ...      ...
2468  Topic20     6.854826       historic    10.343939   2.8285  -5.4054
2114  Topic20     6.611851          zakat     9.960199   2.8302  -5.4414
4083  Topic20     5.020889          gives     7.589067   2.8268  -5.7167
5149  Topic20     2.351832       believes     3.484170   2.8469  -6.4751
1950  Topic20     6.816766     leadership    14.794587   2.4651  -5.4109
1826  Topic20    

In [15]:
from gensim.models.wrappers import Wordrank as wr

In [22]:
model1 = wr()

In [23]:
wr.train()

TypeError: train() missing 3 required positional arguments: 'wr_path', 'corpus_file', and 'out_name'

In [20]:
wr?

In [371]:
wordRank = Wordrank.train(wr_path = 'C:/programdata/anaconda3/lib/site-packages/gensim/models/wrappers/wordrank',corpus_file='C:/Users/DELL/Documents/ITU/Data Science/Data Science Project/imran_khan_corpus.txt', out_name='wr_model_3')

FileNotFoundError: [WinError 2] The system cannot find the file specified

In [397]:
from gensim.models.wrappers import FastText
model = FastText(sg = 1,window = 5, iter = 5, size = 200)
model.train('C:/Users/DELL/fasttext/fasttext.exe', \
                       corpus_file='C:/Users/DELL/Documents/ITU/Data Science/Data Science Project/imran_khan_corpus.txt')

In [399]:
model.similarity('sharif','imran')

AttributeError: 'NoneType' object has no attribute 'shape'

In [412]:
import gensim
import os
from gensim.models.word2vec import LineSentence
from gensim.models.fasttext import FastText as FT_gensim

In [413]:
lee_train_file = 'C:/Users/DELL/Documents/ITU/Data Science/Data Science Project/imran_khan_corpus.txt'
lee_data = LineSentence(lee_train_file)

In [414]:
model_gensim = FT_gensim(sg = 1, size = 200, iter = 10, window = 6)

In [415]:
model_gensim.build_vocab(lee_data)

# train the model
model_gensim.train(lee_data, total_examples=model_gensim.corpus_count, epochs=model_gensim.iter)

print(model_gensim)

FastText(vocab=1520, size=200, alpha=0.025)


In [445]:
model_gensim.similarity('dhandli','puncture')

0.95920098112368812

In [457]:
from sklearn.cluster import KMeans
import time

start = time.time() # Start time

# Set "k" (num_clusters) to be 1/5th of the vocabulary size, or an
# average of 5 words per cluster
word_vectors = model_gensim.wv.syn0norm
num_clusters = round(word_vectors.shape[0] / 10)

# Initalize a k-means object and use it to extract centroids
kmeans_clustering = KMeans( n_clusters = num_clusters )
idx = kmeans_clustering.fit_predict(word_vectors)

# Get the end time and print how long the process took
end = time.time()
elapsed = end - start
print("Time taken for K Means clustering: ", elapsed, "seconds.")

Time taken for K Means clustering:  2.3382182121276855 seconds.


In [458]:
word_centroid_map = dict(zip( model_gensim.wv.index2word, idx ))

In [459]:
for cluster in range(0,40):
    #
    # Print the cluster number  
    print("\nCluster %d" % cluster)
    #
    # Find all of the words for that cluster number, and print them out
    words = []
    for i in range(0,len(word_centroid_map.values())):
        if( list(word_centroid_map.values())[i] == cluster ):
            words.append(list(word_centroid_map.keys())[i])
    print(words)


Cluster 0
['sc', 'abt', 'stashed', 'debt', 'papers', 'trail', 'asif', 'qatari', 'accts', 'flats', 'icij', 'evaders']

Cluster 1
['crowd', 'crowds']

Cluster 2
['future', 'human', 'quality', 'culture', 'achievement', 'went', 'meet', 'structure', 'student', 'hunt', 'achievements', 'silent']

Cluster 3
["'s", 'will', 'system', 'present', 'bill', 'cj', 'biggest', 'level', 'defend', 'feel', 'dev', 'poverty', 'delib', 'delivering', 'estab', 'management', 'issued', 'flawed', 'rewarded', 'sense', 'fool', 'officer', 'purchase', 'tried', 'kept', 'developing', 'confession', 'develop']

Cluster 4
['killing', 'condolences', 'attacked', 'attacking']

Cluster 5
['given', 'it', 'efforts', 'illegal', '+', 'latest', 'basic', 'resources', 'example', 'town', 'because', 'gullu', 'ig', 'ahmed', 'prepared', 'arrested', 'pres', 'basis', 'phc', 'determined', 'challenged', 'preserve', 'largest', 'traffic']

Cluster 6
['force', 'polio']

Cluster 7
['a', 'they', 'brought', 'overseas', 'but', 'social', 'u', 'go',

In [463]:
model_gensim.wv.

array([[-0.13412713, -0.12358898, -0.05287329, ..., -0.16823585,
         0.05334451,  0.19909415],
       [-0.11571744, -0.12180761, -0.05369657, ..., -0.11190248,
         0.02807045,  0.1481328 ],
       [-0.14703797, -0.10617617,  0.01623172, ..., -0.16869846,
         0.05382176,  0.18245675],
       ..., 
       [-0.1516401 , -0.1105182 , -0.07041201, ..., -0.1509161 ,
         0.0590104 ,  0.16276798],
       [-0.14862807, -0.1098968 , -0.05962089, ..., -0.15286499,
         0.06458976,  0.16370949],
       [-0.16002686, -0.10681472, -0.07913794, ..., -0.17251711,
         0.07854193,  0.16034797]], dtype=float32)

In [464]:
import numpy as np

# Carrying out Singular Value Decomposition

U, s, Vh = np.linalg.svd(model_gensim.wv.syn0,full_matrices=False) 

In [473]:
words = model_gensim.wv.index2word

In [474]:
words

['amp',
 "'s",
 'govt',
 'pti',
 'kp',
 'pak',
 'i',
 'pmln',
 'police',
 'ppl',
 'sharif',
 'pakistan',
 'today',
 'r',
 'people',
 'ns',
 'money',
 'corruption',
 'want',
 'rs',
 'pm',
 'nation',
 'sharifs',
 'punjab',
 'we',
 'azadi',
 'workers',
 'attack',
 'public',
 'condemn',
 'time',
 'na',
 'peshawar',
 'children',
 'justice',
 'ag',
 'esp',
 'thank',
 'not',
 'democracy',
 'cricket',
 'nawaz',
 'day',
 'team',
 'report',
 'tax',
 'naya',
 'family',
 'rigging',
 'sc',
 'fight',
 'jalsa',
 'death',
 'state',
 'prayers',
 'party',
 'election',
 'pakistanis',
 'come',
 'great',
 'law',
 'ecp',
 'support',
 'change',
 'new',
 'this',
 'abt',
 'political',
 'women',
 'khan',
 'exposed',
 'power',
 'panama',
 'commitment',
 'record',
 'he',
 'shows',
 'stand',
 'elec',
 'karachi',
 'security',
 'india',
 'a',
 'poor',
 'sindh',
 'peace',
 'media',
 'committed',
 'the',
 'despite',
 'cm',
 'us',
 'families',
 'incl',
 'innocent',
 'lives',
 'world',
 'terror',
 'future',
 'protest',


In [479]:
import matplotlib.pyplot as plt
%matplotlib
# Plotting words and their vector representations

for i in range(len(words)):
        fig = plt.gcf()
        fig.set_size_inches(18.5, 10.5)
        plt.text(U[i,0], U[i,1], words[i])
        plt.xlim((-0.5,0.5))
        plt.ylim((-0.5,0.5))



Using matplotlib backend: Qt5Agg


In [5]:
from sklearn.feature_extraction.text import CountVectorizer

In [6]:
vec = CountVectorizer()
X = vec.fit_transform(lines)

In [7]:
len(vec.get_feature_names())

7743

In [565]:
X.shape

(3233, 7783)

In [572]:
X.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [8]:
import vis_topic as vt
import corex_topic as ct
import scipy.sparse as ss
import matplotlib.pyplot as plt

C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [9]:
doc_word = ss.csr_matrix(X)


In [10]:
import numpy as np

In [11]:
words = list(np.asarray(vec.get_feature_names()))

In [12]:
not_digit_inds = [ind for ind,word in enumerate(words) if not word.isdigit()]
doc_word = doc_word[:,not_digit_inds]
words    = [word for ind,word in enumerate(words) if not word.isdigit()]

doc_word.shape # n_docs x m_words

(3233, 7701)

In [13]:
topic_model = ct.Corex(n_hidden=50, words=words, max_iter=200, verbose=False, seed=None)
topic_model.fit(doc_word, words=words);

In [39]:
# Print a single topic from CorEx topic model
topic_model.get_topics(topic = 1, n_words = 10)

[('idps', 0.024508058272540139),
 ('projects', 0.024214577209497375),
 ('metro', 0.012450094443200315),
 ('wonder', 0.012446136718557417),
 ('nwa', 0.011286696585915671),
 ('pia', 0.010795948650004593),
 ('crisis', 0.0098677754060777102),
 ('hydel', 0.0097611300278441001),
 ('no', 0.008647888535657974),
 ('privatisation', 0.0082882239602527844)]

In [31]:
 x = topic_model.get_topics(topic = 5, n_words = 10)

In [14]:
vt.vis_rep(topic_model, column_label=words, prefix='twenty')

Print topics in text file
